In [1]:
import sys
sys.path.append('..')
from ipywidgets import Textarea, VBox, Button, Output, IntProgress, Layout, Dropdown
from IPython.display import display, clear_output, HTML
import time
import traceback
from src.core import process_order_request, process_ride_plan_request

# --- Define User Queries for Both Domains ---
tax_queries = [
    "We had two burgers at 10 euros each and a large soda for 3 euros.",
    "My order was one salad for 12 euro and a bottle of water for 4 euro."
]
cycling_queries = [
    "I want to plan a long, flat ride. Let's say 100 kilometers.",
    "Let's plan a challenging 50km ride through some hilly terrain.",
    "I'm training for the mountains, plan a 75km ride with mountainous terrain."
]
print("✅ System ready. Two domains are configured.")

✅ System ready. Two domains are configured.


In [2]:
def create_multi_domain_ui():
    domain_selector = Dropdown(
        options=['Tax Calculation', 'Cycling Ride Planner'],
        value='Tax Calculation',
        description='Select Domain:',
        style={'description_width': 'initial'}
    )
    query_input = Textarea(
        value=tax_queries[0],
        description='User Query:',
        style={'description_width': 'initial'},
        layout={'width': '95%', 'height': '100px'}
    )
    run_button = Button(description='Generate DSL & Execute', button_style='success', icon='play')
    progress_bar = IntProgress(value=0, min=0, max=10, description='Waiting...', bar_style='info', orientation='horizontal', layout={'visibility': 'hidden'})
    output_area = Output()

    def on_domain_change(change):
        if change['new'] == 'Tax Calculation':
            query_input.value = tax_queries[0]
        else:
            query_input.value = cycling_queries[0]
    domain_selector.observe(on_domain_change, names='value')

    def run_analysis(button):
        with output_area:
            clear_output(wait=True)
            run_button.disabled = True
            progress_bar.description = 'LLM processing...'; progress_bar.layout.visibility = 'visible'; progress_bar.value = 5

            domain = domain_selector.value
            query = query_input.value

            result = process_order_request(query) if domain == 'Tax Calculation' else process_ride_plan_request(query)

            progress_bar.value = 10
            if result["status"] == "success":
                display(HTML("<h4>1. LLM-Generated DSL Code</h4>"))
                display(HTML(f"<pre style='background-color:#f5f5f5; border: 1px solid #ccc; padding: 10px; border-radius: 5px;'>{result['llm_generated_dsl']}</pre>"))

                display(HTML("<h4>2. Interpreter Execution Result</h4>"))
                dsl_result = result['interpreter_result']

                if domain == 'Tax Calculation':
                    display(HTML(f"""
                    <table style="width:100%; border-collapse: collapse;">
                      <tr style="background-color:#f2f2f2;"><th style="text-align:left; padding:8px; border: 1px solid #ddd;">Component</th><th style="text-align:right; padding:8px; border: 1px solid #ddd;">Calculated Value</th></tr>
                      <tr><td style="padding:8px; border: 1px solid #ddd;">Net Food Cost</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['net_food_cost']:.2f}</td></tr>
                      <tr><td style="padding:8px; border: 1px solid #ddd;">Net Drink Cost</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['net_drink_cost']:.2f}</td></tr>
                      <tr><td style="padding:8px; border: 1px solid #ddd;">Total Tax</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['total_tax']:.2f}</td></tr>
                      <tr style="font-weight:bold; background-color:#f2f2f2;"><td style="padding:8px; border: 1px solid #ddd;">FINAL BILL</td><td style="text-align:right; padding:8px; border: 1px solid #ddd;">€{dsl_result['final_bill']:.2f}</td></tr>
                    </table>"""))
                else: # Cycling Domain
                    display(HTML(f"""
                    <p>The interpreter calculated the following based on the DSL:</p>
                    <ul>
                        <li><b>Distance:</b> {dsl_result['input_properties'].get('distance_km', 0)} km</li>
                        <li><b>Terrain:</b> {dsl_result['input_properties'].get('terrain', 'N/A')}</li>
                        <li style="font-weight:bold;">Estimated Duration: {dsl_result['estimated_duration_hours']:.2f} hours</li>
                    </ul>
                    """))
            else:
                print(f"🔴 PROCESSING ERROR: {result['message']}")

            run_button.disabled = False; progress_bar.layout.visibility = 'hidden'; progress_bar.value = 0
    run_button.on_click(run_analysis)
    print("--- Multi-Domain DSL Validator ---")
    display(VBox([domain_selector, query_input, run_button, progress_bar, output_area]))

create_multi_domain_ui()

--- Multi-Domain DSL Validator ---
